In [11]:
import requests
from bs4 import BeautifulSoup
from lxml import etree

In [12]:
def soup_func(url):
    header = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.0.0'}
    r = requests.get(url, headers=header)
    return BeautifulSoup(r.text, 'lxml')
def pretiffy(soup):
    return soup.prettify()
def xpath(soup, path):
    return etree.HTML(str(soup)).xpath(path)[0].text

In [13]:
requests.get('https://openlibrary.org/trending/forever', verify=False)


/home/soszaboss/Documents/sonatel/projet/python/scarping/openlib_scarping_with_python/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openlibrary.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>

In [14]:
URL = 'https://openlibrary.org/trending/forever'

soup = soup_func(URL)

# pagination list pages
pagination_links = [link['href'] for link in soup.find_all('a', attrs={'class': 'ChoosePage'})][:-1:]
pagination_links = ['/trending/forever?page=1'] + pagination_links
print(pagination_links)

['/trending/forever?page=1', '/trending/forever?page=2', '/trending/forever?page=3', '/trending/forever?page=4', '/trending/forever?page=5', '/trending/forever?page=6', '/trending/forever?page=7', '/trending/forever?page=8', '/trending/forever?page=9', '/trending/forever?page=10']


In [17]:
for _ in pagination_links:

    URL = f"https://openlibrary.org{_}"

    #scarpe the actual pagination link page
    pagination_soup = soup_func(URL)

    # get the list of books of this actual page
    list_group = pagination_soup.find_all('li', attrs={'class': 'searchResultItem'})

    # insert into a dictionary format books detail
    data = []

    # get the link detail page of every book
    links_detail_book_page = [link.find('a', attrs={'itemprop': 'url'})['href'] for link in list_group]
    for books_detail in links_detail_book_page:
        NEW_URL = f"https://openlibrary.org{books_detail}"
        #book_detail = requests.get(NEW_URL)
        new_soup = soup_func(NEW_URL) #BeautifulSoup(book_detail.text, 'html.parser')
        publishers = [publisher.text for publisher in new_soup.find_all('a', attrs={'itemprop': 'publisher'})]
        try:
            isbn = [isbn.text for isbn in new_soup.find_all('dd', attrs={'class': 'object', 'itemprop': 'isbn'})]
            isbn10 = None
            isbn15 = None
        except:
            isbn10 = isbn[0]
            isbn15 = isbn[1]
        try:
            page = new_soup.find('span', attrs={'class':'edition-pages' ,'itemprop': 'numberOfPages'}).text
        except:
            page = None
        try:
            language = [language.a.text for language in new_soup.find_all('span', attrs={'itemprop':'inLanguage'})]
        except ValueError:
            language = None
        try:
            title = new_soup.find('h1', attrs={'class':'work-title', 'itemprop':'name'}).text
        except:
            title = None
        book_data = {
                'title': title,
                'author': new_soup.find('a', attrs={'itemprop': 'author'}).text, #xpath(new_soup, '//*[@id="contentBody"]/div[1]/div[3]/div[2]/span/h2[2]/a'),
                'publication_date': xpath(new_soup, '//*[@id="contentBody"]/div[1]/div[3]/div[5]/div/div[1]/span'),
                'page': page,
                'language': ' '.join(language),
                'publishers': f"{', '.join(publishers)}",
                'description': xpath(new_soup, '//*[@id="contentBody"]/div[1]/div[3]/div[4]/div/p[1]'),
                'isbn10': isbn10, #xpath(new_soup, '//*[@id="contentBody"]/div[1]/div[3]/div[9]/div[4]/dl/dd[4]'),
                'isbn15': isbn15,
                'image':new_soup.find('img', attrs={'itemprop': 'image'})['src'],
                'genre':[a.text for a in new_soup.find_all('a', attrs={'data-ol-link-track' : "BookOverview|SubjectClick"})]
                }
        data.append(book_data)
        print(book_data)
    print(data)

{'title': 'It Ends With Us', 'author': 'Colleen Hoover', 'publication_date': 'August 2nd, 2016', 'page': '384', 'language': 'Portuguese', 'publishers': 'Simon & Schuster UK Ltd., Simon & Schuster Ltd', 'description': 'Lily hasn’t always had it easy, but that’s never stopped her from working hard for the life she wants. She’s come a long way from the small town where she grew up—she graduated from college, moved to Boston, and started her own business. And when she feels a spark with a gorgeous neurosurgeon named Ryle Kincaid, everything in Lily’s life seems too good to be true.\n', 'isbn10': None, 'isbn15': None, 'image': 'https://archive.org/download/itendswithus0000hoov/page/cover_w180_h360.jpg', 'genre': ['Fiction, Romance, Contemporary, Life change events', 'Triangles (Interpersonal relations)', 'Man-woman relationships', 'Neurosurgeons', 'First loves', 'Businesswomen', 'Fiction', 'Contemporary Women', 'New York Times bestseller', 'Fiction, romance, contemporary', 'Man-woman relati